## AWS Client Setup [local]

Installa la AWS CLI scaricandola da questo url: https://docs.aws.amazon.com/cli/latest/userguide/getting-started-install.html#getting-started-install-instructions

Una volta installata, incolla le informazioni dalla Command line or programmatic access nella folder .aws nel profilo utente di sistema.

Nel file config
```json
[username]
region = us-west-2
output = json
```

Nel file credential inserire le informazioni in questo formato:

```json
[username]
aws_access_key_id=
aws_secret_access_key=
aws_session_token=
```

In [26]:

import os
os.environ["AWS_DEFAULT_REGION"] = "us-west-2"  # E.g. "us-east-1"
os.environ["AWS_PROFILE"] = "[fillthis]"

In [45]:
import os, sys
from utils import bedrock
from langchain.llms import Bedrock


bedrock_runtime = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
    runtime=True
)

model = Bedrock(
    model_id="anthropic.claude-v2", 
    client=bedrock_runtime,
    model_kwargs={'temperature': 0.5}
    )

Create new client
  Using region: us-west-2
  Using profile: sky
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-west-2.amazonaws.com)


# AWS Client Setup [AWS LAB]

In [ ]:
import json
import os
import sys

import boto3
import botocore

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww

# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."


bedrock_runtime = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
    runtime=True
)

### Test del client

In [29]:
model.invoke("Componi una poesia sulla città di Napoli")

" Ecco la mia poesia sulla città di Napoli:\n\nNapoli bella, città del sole\nTra il mare blu e il Vesuvio che tuona\nLe tue strade piene di vita e rumore\nIl tuo dialetto che scalda il cuore\n\nPizza margherita, sfogliatella e babà  \nProfumi e sapori che incantano già\nMandolini e canzoni napoletane\nRiempiono l'aria di note lontane\n\nIl tuo golfo splendido circonda  \nLa tua storia antica e profonda\nTra vicoli stretti e palazzi maestosi\nNapoli mia, sei proprio preziosa\n\nCon arte e cultura ti distingui\nE un calore umano che sempre distingui\nNapoli amata, Napoli mia\nSei tu la regina della melodia"

## Langchain Interface

Langchain mette a disposizione una interfaccia agnostica rispetto al LLM che decidiamo di utilizzare. 
I metodi utilizzabili sono:
- stream
- invoke
- batch

Ci sono anche i corrispondenti metodi async che saranno noti a chi conosce python. Per ora li lasceremo da parte.

Questi metodi possono essere utilizzati tramite il LangChain Expression Language (LCEL)


In [30]:
from langchain_core.prompts import ChatPromptTemplate

In [31]:
prompt = ChatPromptTemplate.from_template("Componi una poesia sulla seguente città: {city}")
chain = prompt | model

In [32]:
messages = [
    ("system","Sei un cantante adori comppor canzoni ma odi comporre poesie. Se l'utente ti chiede di comporre poesie rispondi in modo sgarbato"),
    ("human","Componi una poesia riguardo la seguente città: {city}")
]
prompt = prompt = ChatPromptTemplate.from_messages(messages=messages)
chain = prompt | model

In [33]:
chain.input_schema.schema()

{'title': 'PromptInput',
 'type': 'object',
 'properties': {'city': {'title': 'City', 'type': 'string'}}}

In [34]:
chain.output_schema.schema()

{'title': 'BedrockOutput', 'type': 'string'}

### Stream
Invoca la generazione e ritorna i chunck man mano che sono generati

In [35]:
messages = [
    ("system","Sei poeta che si ispira al Dolce Stil Novo"),
    ("human","Componi una poesia riguardo la seguente città: {city}")
]
prompt = prompt = ChatPromptTemplate.from_messages(messages=messages)
chain = prompt | model

In [36]:
for s in chain.stream({"city": "Biella"}):
    print(s, end="", flush=True)

 Ecco la mia umile proposta di poesia in stile Dolce Stil Novo su Biella:

O nobile città di Biella bella
che sorgi tra le Alpi a testa alta, 
le tue vie lastricate di pietra arenaria
conducono al tuo cuore antico.

Le acque del Cervo e dell'Oropa
ti bagnano delicatamente,
mentre il Piazzo ti sovrasta
con le sue torri medievali.

I tuoi telai da secoli tessono
preziose stoffe di lana e cachemire,
frutto dell'ingegno e del lavoro
di generazioni di abili artigiani.

Tra i palazzi nobiliari
si affacciano cortili fioriti,
che profumano di gelsomino e rosmarino
in questa primavera ormai imminente.

O Biella, città operosa e gentile,
accoglimi tra le tue

### Invoke
Invoca LLM con i parametri inviati e restituisce la risposta solo a fine elaborazione

In [37]:
chain.invoke({"city": "Milano"})

" Eccomi, nobile poeta, a comporre versi in stile dolce e nuovo su Milano:\n\nO Milano gentile, città industriosa\nche sorgi altera in mezzo alla pianura.  \nLe tue torri svettano verso il cielo, \nsimbolo di operosità e ingegno.\n\nLe vie brulicano di gente indaffarata,  \nmentre il Duomo veglia maestoso e antico.\nTra i palazzi e le piazze si respira \nun'aria fresca di novità e progresso. \n\nO Milano, tu sei fucina di talenti,\ncrocevia di culture, culla di bellezza.\nNel tuo grembo fervono arte e scienza,\ne tutto il mondo ammira la tua grandezza.\n\nCon affetto ti canto, città laboriosa,\nche guardi al futuro con passo sicuro.\nRicca di storia, eppur sempre in movimento,"

### Batch
Permette di invocare la stessa chain in parallelo su parametri diversi

In [38]:
chain.batch([{"city": "Napoli"}, {"city": "Roma"}])

[" Ecco una poesia in stile Dolce Stil Novo su Napoli:\n\nNapoli mia, città del cuore\nDove amore e passione ardono in core.\nTra vicoli stretti e antichi palazzi  \nSi respira profumo di sfogliatelle e mozzarelle.\n\nIl tuo golfo blu come zaffiro\nRiflette il sole e il tuo bel sguardo.\nIl Vesuvio ti veglia da lontano\nSimbolo del tuo animo vulcanico.\n\nLa tua voce è musica pura  \nChe riempie l'aria di gioia e dolcezza.\nSei tu la sirena che mi ha stregato\nNapoli, sei tu la mia felicità.",
 " Eccomi, signora mia, a comporre per voi una poesia nello stile del Dolce Stil Novo, dedicata alla città eterna di Roma:\n\nO nobile Roma, città senza tempo, \nche nel tuo grembo racchiudi tesori,\ndi gloria e potere un giorno fosti centro,\nora dimora d'artisti e sognatori.\n\nLe tue vie lastricate parlano al cuore,\ni Fori e il Colosseo narrano gesta antiche,\nla luce del tramonto dona calore\nalle tue pietre, mute ma loquaci.\n\nFontana di Trevi, dove l'acqua scorre\ne con le sue cascate inc

### Parallelism

In [39]:
from langchain_core.runnables import RunnableParallel

In [40]:
chain1 = ChatPromptTemplate.from_template("Componi una poesia su {topic}") | model
chain2 = (
    ChatPromptTemplate.from_template("Scrivi un possibile slogan per pubblicizzare i giochi olimpici nella città di {topic}")
    | model
)
combined = RunnableParallel(poem=chain1, slogan=chain2)

In [41]:
combined.invoke({"topic": "Napoli"})

{'poem': ' Ecco la mia poesia su Napoli:\n\nNapoli bella, città del sole  \nTra il mare e il Vesuvio, regina del golfo  \nLe tue strade brulicano di vita  \nI tuoi vicoli raccontano storie\n\nPizza margherita, profumo di sfogliatelle  \nVoci che si rincorrono tra i quartieri  \nIl caffè bollente scalda i cuori  \nLa gente si incontra, chiacchiera e ride\n\nMare blu cobalto, lungomare affollato  \nIl sole cocente scalda la pietra  \nGli scugnizzi giocano tra i vicoli  \nLe nonne cuciono sul balcone  \n\nNapoli mia, quanta vita pulsante  \nTra luci e ombre, gioia e dolore  \nCittà passionale, dal cuore immenso  \nNapoli bella, ti porto nel cuore',
 'slogan': ' Ecco una proposta di slogan per pubblicizzare i Giochi Olimpici a Napoli:\n\n"Napoli 20XX, dove lo sport incontra la passione"'}

### Parallelism on batch

In [42]:
combined.batch([{"topic": "Roma"}, {"topic": "Milano"}])

[{'poem': ' Ecco una poesia su Roma composta da me:\n\nRoma eterna, città millenaria\nDalle rovine parlanti di storia antica\nLe vie lastricate risuonano i passi\nDi imperatori, poeti e santi\n\nIl Colosseo svetta maestoso \nTestimone di lotte e di gloria\nFontana di Trevi luccica al chiaro di luna\nDesideri esauditi per chi vi getta una moneta\n\nLa Città del Vaticano e la Basilica di San Pietro\nCuore pulsante della cristianità  \nPiazza Navona gremita di turisti\nChe ammirano Bernini nel suo capolavoro\n\nRoma, culla di civiltà e cultura\nOgni angolo nasconde un tesoro\nLa tua bellezza supera i secoli\nGioiello senza tempo di storia e di arte',
  'slogan': ' Ecco una proposta di slogan per pubblicizzare i Giochi Olimpici a Roma:\n\n"Roma 2024, dove la storia incontra il futuro. Vivi l\'emozione delle Olimpiadi nella Città Eterna!"'},
 {'poem': ' Ecco la mia poesia su Milano:\n\nMilano, città della moda e del design\nTra grattacieli che sfiorano il cielo\nVivi e frenetica, sempre in 

## Chat Interface

In [46]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory


In [47]:
memory = ConversationBufferMemory(return_messages=True)

In [50]:
conversation = ConversationChain(
    llm=model,
    verbose=True,
    memory=ConversationBufferMemory()
)

In [52]:
conversation.predict(input="Nice to meet you. How many km are there between Earth and Mars?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Nice and from the sun?
AI:  I'm afraid I don't have enough context to understand what you mean by "Nice and from the sun." Could you please rephrase or provide more details about what you're asking? I'd be happy to try answering again with some more context.
Human: Nice to meet you. How many km are there between Earth and Mars?
AI:


c:\Users\EGRIMAAY3\OneDrive - NTT DATA EMEAL\Desktop\antonio\progetti\2023\sky\workshop\aws-workshop-genai\.venv\Lib\site-packages\langchain_community\llms\bedrock.py:73: UserWarning: Error: Prompt must alternate between '

Human:' and '

Assistant:'. Received The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Nice and from the sun?
AI:  I'm afraid I don't have enough context to understand what you mean by "Nice and from the sun." Could you please rephrase or provide more details about what you're asking? I'd be happy to try answering again with some more context.

Human: Nice to meet you. How many km are there between Earth and Mars?
AI:

Assistant:
  warnings.warn(ALTERNATION_ERROR + f" Received {input_text}")



> Finished chain.


' There is an average distance of about 225 million km (140 million miles) between Earth and Mars. The actual distance varies as the planets orbit the Sun, but this is the approximate mean distance.\n\nSome key facts about the distance between Earth and Mars:\n\n- The minimum distance is around 54.6 million km when Mars is closest to Earth (at opposition). \n\n- The maximum distance is about 401 million km when the planets are farthest apart.\n\n- When Mars is closest to Earth, it takes a radio signal 3 minutes to travel between the planets. At the farthest point, it takes about 22 minutes.\n\n- The planets orbit the Sun at different speeds, so the distance is constantly changing over time.\n\n- Spacecraft launching from Earth to Mars take anywhere from 6-9 months for the journey, depending on the specific trajectory.\n\nSo in summary, the average distance is about 225 million km, but it does vary between around 55 million km at the closest point to 401 million km at the farthest point

In [54]:
conversation.predict(input="what distance from the sun?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Nice and from the sun?
AI:  I'm afraid I don't have enough context to understand what you mean by "Nice and from the sun." Could you please rephrase or provide more details about what you're asking? I'd be happy to try answering again with some more context.
Human: Nice to meet you. How many km are there between Earth and Mars?
AI:  There is an average distance of about 225 million km (140 million miles) between Earth and Mars. The actual distance varies as the planets orbit the Sun, but this is the approximate mean distance.

Some key facts about the distance between Earth and Mars:

- The minimum distance is around 54.6 million km when Mars is closest to Earth

c:\Users\EGRIMAAY3\OneDrive - NTT DATA EMEAL\Desktop\antonio\progetti\2023\sky\workshop\aws-workshop-genai\.venv\Lib\site-packages\langchain_community\llms\bedrock.py:73: UserWarning: Error: Prompt must alternate between '

Human:' and '

Assistant:'. Received The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Nice and from the sun?
AI:  I'm afraid I don't have enough context to understand what you mean by "Nice and from the sun." Could you please rephrase or provide more details about what you're asking? I'd be happy to try answering again with some more context.

Human: Nice to meet you. How many km are there between Earth and Mars?
AI:  There is an average distance of about 225 million km (140 million miles) between Earth and Mars. The actual distance varies as the planets or


> Finished chain.


" Based on the context of our previous discussion about the distance between Earth and Mars, I assume you are now asking about the average distances of Earth and Mars from the Sun.\n\nThe average distances from the Sun are:\n\n- Earth: 149.6 million km (92.96 million miles)\n\n- Mars: 227.9 million km (141.6 million miles) \n\nSome key facts:\n\n- Earth's distance from the Sun varies from 147.1 million km at perihelion (closest point) to 152.1 million km at aphelion (farthest point).\n\n- Mars' distance from the Sun varies from 206.7 million km at perihelion to 249.2 million km at aphelion.\n\n- The orbits of both planets are elliptical rather than perfect circles.\n\n- Earth is closer to the Sun than Mars is on average. This is why Earth is warmer and Mars has a colder climate.\n\nPlease let me know if you need any clarification or have additional questions! I'm happy to provide more details about the distances between the Sun, Earth and Mars."

# Chatbot with Role

In [55]:
memory = ConversationBufferMemory()
memory.chat_memory.add_user_message("Context:You will be acting as an expert fitness coach. Your job is to give advice and plan training schedules to help users reach their fitness goals.")
memory.chat_memory.add_ai_message("I am an expert fitness coach and give gym advice")
conversation = ConversationChain(
    llm=model,
    verbose=False,
    memory=ConversationBufferMemory()
)

In [57]:
print(conversation.predict(input="My goal is to increase my strenght and compete in a powerlifting competition. Can you help me with my workout?"))

 Here are some tips to help you train for and compete in a powerlifting competition:

- Focus on the three main lifts - squat, bench press, and deadlift. Make sure your technique on these lifts is solid before adding a lot of weight. Work on form with lighter weights first.

- Follow a program specifically designed for powerlifting. Look for ones that focus on progressively overloading the main lifts with heavy weights and lower reps. Many programs will also incorporate accessory exercises to build overall strength. 

- Train the main lifts at least twice per week. Squat and deadlift 1-2 times per week each. Bench press can be trained 2-3 times per week. 

- Include accessory exercises like overhead press, rows, pull ups, dips, lunges, etc. These help build overall strength to support the main lifts.

- Do most sets in the 1-5 rep range on the big lifts. This builds strength and power. Higher reps can be used on accessory lifts.

- Take sufficient rest between sets (3-5 mins) when lift

In [58]:
print(conversation.predict(input="Ok, but what about progression in every lift week to week?"))

c:\Users\EGRIMAAY3\OneDrive - NTT DATA EMEAL\Desktop\antonio\progetti\2023\sky\workshop\aws-workshop-genai\.venv\Lib\site-packages\langchain_community\llms\bedrock.py:73: UserWarning: Error: Prompt must alternate between '

Human:' and '

Assistant:'. Received The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: My goal is to increase my strenght and compete in a powerlifting competition. Can you help me with my workout?
AI:  Here are some tips to help you train for and compete in a powerlifting competition:

- Focus on the three main lifts - squat, bench press, and deadlift. Make sure your technique on these lifts is solid before adding a lot of weight. Work on form with lighter weights first.

- Follow a program specifically designed for powerlifting. Look for ones that focus on

 Here are some tips on weekly progression for powerlifting training:

- Increase weight on the bar gradually each week to drive progression. Aim for 2.5-10 lbs each week on main lifts.

- Use submaximal weights for multiple sets of 1-5 reps. For example, do 3 sets of 3 reps at around 85% of your 1 rep max. Add weight weekly.

- Alternate heavy and light days for the main lifts. Heavy days use heavier weights for lower reps. Light days use lighter weights for 5-8 reps. 

- On light days, focus on speed and technique over weight lifted. This allows recovery from heavy days.

- Periodize your training into mesocycles of 4-8 weeks. Progressively increase weight each mesocycle while lowering volume. Then reset with higher volume and lower weights.

- Incorporate periodized or programmed deload weeks every 4-8 weeks to allow recovery. Reduce volume and intensity during these weeks. 

- Monitor rate of progression each week and adjust as needed. If lifts stall for 2-3 weeks, try changing rep 